In [2]:
#!/usr/bin/env python3
# kgmcdaid

#sebowman provided feedback and input on the method execution, code styling, user testing, specializing code for the desired audience, and more. 

"""
Title: fileCleaning_userfriendly.ipynb
Author: kgmcdaid


This has been built using the ACS* 2017 5-year Estimates of the state of Georgia as inputs.
It is assumed from the notes file provided with downloaded ACS data that conventions are consistent across states.


Note: This jupyter notebook has been developed with the intention of being usable by those with minimal 
experience coding to use text inputs within the jupyter notebook to specify the route by which the files are processed. 
Aside from some individual methods, user inputs are required.


Input:
    inFile, an ACS csv file containing education, poverty or plumbing data. 
    If it contains a different set of ACS data, can designate as 'general,
    and a more limited set of methods will be performed. 
    
    Uses text inputs from user. (Optionally) may use some inputs given during the instantiation of the class. 
    User (text) input is always required for: 
        csv input/output pathname,
        the booleans where default is False, unless instantiated as True, which are niceName, add, desc. 



Output: 
    A csv file containing the processed input file.
    Optionally, a description file containing statistics for each column. 

TODO: 
    changeAll() TODO:
    Convert to null: implement solutions where all but those which can be converted 
    to float are replaced with null. 
    This would be a flipped version where the replaced values are not hardcoded in. 
    Similar to what is discussed here: 
    https://stackoverflow.com/questions/1450897/remove-characters-except-digits-from-string-using-python

    eduData() TODO:  
    Make drop columns customizable. 
    In the education dataset, some columns have the entry '2,500-'.
    This does not have an interpretation provided in the ACS notes file. 
    As such, these are marked as null.  
    If drop=True, the columns that contain them are dropped regardless.

    checkRows() TODO:
    Could add another that triggers if checkNull > 0, where it removes the values.
    
    readIn()
    Cannot be currently passed from main. 

    main() TODO:
    Ability to choose between using text inputs or Argparse. 

Resources/references:
    https://stackoverflow.com/questions/26347412/drop-multiple-columns-in-pandas
    https://stackoverflow.com/questions/48158688/save-pandas-describe-for-human-readibility
    https://github.com/pandas-dev/pandas/issues/14086

    https://realpython.com/documenting-python-code/
    https://www.python.org/dev/peps/pep-0257/
    
* ACS = American Community Survey, data.census.gov or factfinder.census.gov (archival)

"""

import pandas as pd
import numpy as np

class ACSdata():
    """ Process the American Community Survey Data according to user inputs. 
    
    Packages: numpy, pandas 
    
    This processes ACS data according to its type (poverty- pov, education- edu, plumbing- plu),
    or more generally.
    
    When instantiating, can call newFile() to run all methods in the class. 
    
    Methods: 
        newFile() reads in the csv file and call all the methods that don't depend on dataType().
        dataType() calls methods that process according to dataset type.
        povData() removes unnecessary columns from the poverty dataset. calls changeAll, povHeaders.
        eduData() removes unnecessary columns from the education dataset. calls changeAll, eduHeaders.
        pluData() removes unnecessary columns from the plumbinh dataset. calls changeAll, pluHeaders.
        
        changeAll() processes dataframe to have only a single datatype in each column.
        checkRows() sums null values by row, calls removeNullRows.
        removeNullRows() removes rows with high null value counts.
        checkCols() sums null values by column.
        
        readIn() checks if the file read in has a valid filename. 
        addCounty() if add is True, adds column 'County' containing county from GEO.display-label (renamed 'Geography')
        describe() generates descriptive statistics file.
        whatOut() specifies filename, path for output file.
        
        headers() changes headers for the columns with headers GEO.id2 and GEO.display-label, 
            gives option to change other headers according to dataset type.
        povHeaders() changes some headers for pov dataset for more readable labels.
        eduHeaders() changes some headers for edu dataset for more readable labels.
        pluHeaders() changes some headers for plu dataset for more readable labels.
        
    Note: dataset type is the type of ACS data- pov, edu, plu, or a general ACS dataset. 
    The general designation performs all methods that are not specific to a dataset type. 
    
    ACS_17_5YR_S1501, the education dataset for Georgia has inconsistent descriptions, specifically for 
    HC02_EST_VC17 -- 
    Metadata csv: Percent Estimate, Percent high school graduate or higher
    Data with annotations csv: Population 18 to 24 years - Bachelor's degree or higher

    """

    def __init__(self, 
                 data=None, 
                 niceName=False, 
                 add=False, 
                 drop=True, 
                 desc_file=None
                ):
        """ Initialize drop as True; initialize all other parameters as None, False or empty list.
        
        optional parameters:
            inFile: None
            df: None
            data: None
            niceName: bool, default False
            drop: bool, default True
            add: bool, default False
            noneListR: list
            desc: False 
        
        """
        self.df = None
        self.data = data
        self.niceName = niceName 
        self.add = add
        self.drop = drop
        self.noneListR = []
        self.noneListC = []
        self.dropCol = []
        self.out = None
        

    def newFile(self):
        """ Read in the csv file and call all the methods involved in the processing of the file that are independent of the input to dataType().
        
        Called during instantiation. 
        Print confirmation of the output file name. 
        """
        self.readIn() # check if valid file
        self.df = pd.read_csv(self.inFile)
        self.headers()
        self.dataType() #process according to type
        self.checkRows()
        self.checkCols()
        self.addCounty()
        self.describe()
        self.whatOut()
        self.df.to_csv(self.out)
        print('Look in directory for the output file {}'.format(self.out))
        #print('newFile done')
       
    def dataType(self):
        """ Designate the type of ACS dataset. If invalid, prompt again.
        
        Arguments: 
            data : str - input('which ACS dataset? enter pov, edu or plu: ') or passed from main().

        Valid dataset types, meanings: 
            pov = poverty dataset
            edu = education dataset
            plu = plumbing dataset
            gen = a general ACS dataset
        """
        
        if self.data == None: 
            self.data = input('which ACS dataset? enter pov, edu,  plu or gen: ')
        print('dataType: {}'.format(self.data))
        # if self.data == 'pov':
        if self.data == 'pov':
            self.n = 15
            print('Processing ACS poverty dataset')
            self.povData()
        #elif self.data == 'edu':
        elif self.data == 'edu':
            self.n = 15
            print('Processing ACS education dataset')
            self.eduData()
        # elif self.data == 'plu':
        elif self.data == 'plu':
            self.n = 5
            print('Processing ACS plumbing dataset')
            self.pluData()
        elif self.data == 'gen' or 'general': 
            self.n = 15
            changeAll()
        else:
            print('Invalid entry')
            self.data = None
            self.dataType()

            
    # Branching methods
    
    def povData(self):
        """ Remove unnecessary columns from the poverty dataset. 
        
        Drop the columns after column 75. Call ChangeAll. If niceName is True, call povHeaders. 
        Assumes the complete poverty table is entered.
        """
        
        self.df = self.df.drop(self.df.iloc[:,75:None], axis=1)
        self.changeAll()
        if self.niceName == True:
            self.povHeaders()
        # print('povData done')
        
    def eduData(self):
        """ Remove unnecessary columns from the education dataset.
        
        Drop the columns after column 184. 
        If drop == True, Drop columns containing 'VC02', 'VC08', 'HC01', 'HC03', 'HC05'
        Call ChangeAll. If niceName is True, call eduHeaders.
        Adds column 'min_HS_25', the summed values of columns of those over 25 with at least HS attainment. 
        Assumes the complete education table is entered.
        
        Arguments: 
            drop: bool. default True.
            drop_col: list
        
        Note: 
        column HC02_EST_VC17 has inconsistent descriptions
        Metadata csv: Percent Estimate, Percent high school graduate or higher
        Data with annotations csv: Population 18 to 24 years - Bachelor's degree or higher
        """    
        
        self.df = self.df.drop(self.df.iloc[:,184:None], axis=1)
        print(self.drop)
        self.drop_col = ['HC01', 'HC03', 'HC05', 'VC02', 'VC08']
        if self.drop == True:
            for a in self.drop_col:
                self.df = self.df.drop(self.df.columns[self.df.columns.str.contains(a)], axis=1)
            print('Categories containing the following in their headers were dropped: {}'.format(a))
        self.changeAll()
        self.df['min_HS_25'] = self.df['HC02_EST_VC11'] + self.df['HC02_EST_VC12'] + self.df['HC02_EST_VC13'] \
                                + self.df['HC02_EST_VC14'] + self.df['HC02_EST_VC15']
        #Percent over 25 with min HS attainment = HS diploma + Some College + Associates + Bachelors + Grad school 
        print('min_HS_25, the sum of the columns with the percent over 25 with, at minimum HS attainment, has been added ')
        if self.niceName==True:
            self.eduHeaders()
        # print('eduData done')

    def pluData(self):
        """ Call changeAll, then if niceName is True, call eduHeaders. 
        
        Intended to affect the plumbing columns of the Housing units characteristics dataset
        Calclulate and add to dataframe: 'PLU_P_Lacking' = (Count lacking plumbing / Total)*100. 
        """
        
        self.changeAll()
        if self.niceName == True:
            self.pluHeaders()
            self.df['PLU_P_Lacking'] = (self.df['PLU_Lacking']/self.df['PLU_Total'])*100
        else: 
            self.df['PLU_P_Lacking'] = (self.df['HD01_VD03']/self.df['HD01_VD01'])*100

        
    # Cleanup-related methods
    
    def changeAll(self):   
        """ Process dataframe to have only a single datatype in each column.
        
        Remove the first row (descriptor row) and the first column (GEO.id).
        Replace the symbols denoting excluded values with Null.
        Convert the columns containing the collected numerical data to floats.
        """
        
        self.df.drop(self.df.columns[0], axis=1, inplace=True) 
        self.df.drop(self.df.index[0], axis=0, inplace=True)
        self.df.reset_index=True
        
        float_list = [col for col in self.df.columns]
        float_list = float_list[2:]
        self.df[float_list] = self.df[float_list]\
            .replace('-', np.nan)\
            .replace('**', np.nan)\
            .replace('***', np.nan)\
            .replace('*****', np.nan)\
            .replace('+', np.nan)\
            .replace('(X)', np.nan)\
            .replace('N', np.nan)\
            .replace('2,500-',np.nan)\
            .astype('float') 
        
    def checkRows(self):
        """ Check for rows where the count of null values are greater than a specified number.

        Input:
            self.n = int, the threshold of null values
        
        Print:
            Statement with count of rows before removing those above the threshold.
            Statement with count of columns containing >n null values
        """
        
        print("Before removing null values, the length of the dataset is {} rows".format(len(self.df)))
        cols = [col for col in self.df.columns]
        rowNone = self.df.isnull().sum(axis=1)
        for index, value in rowNone.items():
            if value >= self.n:
                self.noneListR.append(index)
        print('The {0} rows with >{1} null are: {2}'.format(len(self.noneListR), self.n, self.noneListR))
        self.removeNullRows()
        
    def checkCols(self, remove=False):
        """ Check for columns where Null values are greater than a specified number.
        
        Input: 
            self.n = int, the threshold of null values
        
        Print:
            Statement with count of columns containing >n null values. 
        """
        
        colNone = self.df.isnull().sum(axis=0)
        for index, value in colNone.items():
            if value >= self.n:
                self.noneListC.append(index)
        print('The remaining {0} columns with >{1} null are: {2}'\
              .format(len(self.noneListC), self.n, self.noneListC))
                                 
    def removeNullRows(self):
        """Removes the rows where nulls are greater than a specified number. Print new length of dataset."""
        noneList_shifted = list(np.array(self.noneListR) - np.array([1 for i in range(len(self.noneListR))])) #sebowman
        xList = self.df.iloc[noneList_shifted, : ]
        self.df = self.df.drop(self.df.index[[noneList_shifted]], axis = 0).reset_index(drop = True)
        print("The length of the dataset is now {} rows".format(len(self.df)))
        
        
    # Customization methods   
    
    def readIn(self):
        """ Obtain the pathname of the input csv file and check whether it is a valid pathname. 
        
        Exception raised: FileNotFoundError: 
        if FileNotFoundError, repeat method.
        If user entered an invalid pathname to the csv file, prompt for input again.
        
        Input:
            inFile: str - input('Enter file name or pathname, including ".csv": ').
            Cannot be currently passed from main. 
        
        Print:
            Confirmation of pathname entered.
        """
        
        self.inFile = input('Enter file name or pathname, including ".csv": ')
        print('Pathname entered: {}'.format(self.inFile))
        try:
            pd.read_csv(self.inFile)
        except FileNotFoundError:
            print('The file {} was not found. Please enter valid csv pathname.'.format(self.inFile))
            self.readIn()  

    def addCounty(self):
        """ If the input is not None, add a column containing the county of each row.
        
        Input: 
            add: bool, default False. If passed from main as False, will be prompted for user input. 
            Press return for False (provide no input).
        
        Print: 
            Confirmation of choice- 'County column added: True/False'
        """
        
        if self.add == False: 
            self.add = bool(input('Add county column? Enter any character for True. Else press return.'))
        if self.add == True:
            geography = self.df.loc[ : , 'Geography']
            county_list = []
            for entry in geography: 
                county_list.append(entry.split(',')[-2])
            counties = np.array(county_list)
            self.df['County'] = counties
        print('County column added: {}'.format(self.add))
        
    def describe(self):
        """ Prompt for whether user wants descriptive file made.
        
        If desc is False, prompt for input.
        If no input to prompt, desc remains False. 
        If True, prompt for the filename, write to file. 
        
        Input:
            desc: bool, default False. If passed from main as False, will be prompted for user input
            desc_file: str
        
        Print:
            Confirmation of choice: 'Generate descriptive file: True/False'
            Confirmation of pathname entered: 'Look in directory for the descriptive file [desc_file]'
        """
        
        """if desc == False:
            desc = bool(input('Generate descriptive file? Enter any character for True. Else press return.'))
        print('Generate descriptive file: {}'.format(self.desc))
        if desc == True:
            desc_file = input('Enter descriptive file name or pathname, including ".csv": ')
            self.df.describe(include='all').to_csv(desc_file)
            print('Look in directory for the descriptive file {}'.format(desc_file))""" 
            
        desc = bool(input('Generate descriptive file? Enter any character for True. Else press return.'))
        print('Generate descriptive file: {}'.format(desc))
        if desc == True:
            desc_file = input('Enter descriptive file name or pathname, including ".csv": ')
            if desc_file == '':
                desc_file = None
                self.describe()
            self.df.describe(include='all').to_csv(desc_file)
            print('Look in directory for the descriptive file {}'.format(desc_file))   
            
            
    def whatOut(self):
        """ Require user to enter a filename for the output if it was not given during initialization. 
        
        If still None, prompt for input again.
        
        Input: 
            out: str, default None - input('Enter output file name or pathname, including ".csv": ') or passed from main()
        """
        
        self.out = input('Enter output file name or pathname, including ".csv": ')
        if self.out == (None or ''):
            self.whatOut()
        
                 
                
    # Header modification methods
    def headers(self): 
        """ Rename the first two columns as 'id' and 'Geography' respectively, provide option to rename other headers.
        
        Rename 'GEO.id2' as 'id', 'GEO.display-label' as 'Geography'.
        If niceName is not instantiated as True, prompt user as to whether to change the headers.
        Depending on data, will change to those in povHeaders, eduHeaders or pluHeaders.
        
        Input: 
            niceName: bool, default False - 
            input('Prefer different headers? Enter any character for True. Else press return.') or passed from main()
        
        Print: 
            Confirmation of choice: True or False.
        """
        
        headers={'GEO.id2':'id',
                 'GEO.display-label':'Geography'}
        self.df.rename(columns=headers, inplace=True)
        if self.niceName == False:
            self.niceName = bool(input('Prefer different headers? Enter any character for True. Else press return.'))
        print('Different headers is {}'.format(self.niceName))        

    def povHeaders(self):
        """Change some of the headers of the plumbing dataset. Called if niceName == True.
        
        Include only headers with the code HC03 (percent poverty, percent poverty margin of error). 
        """
        
        headers = {
        'HC03_EST_VC01':'PPOV',
        'HC03_MOE_VC01':'PPOV_MOE',
        'HC03_EST_VC03':'PPOV_U18',
        'HC03_MOE_VC03':'PPOV_U18_MOE',
        'HC03_EST_VC04':'PPOV_U5',
        'HC03_MOE_VC04':'PPOV_U5_MOE',
        'HC03_EST_VC07':'PPOV_Adult_U64',
        'HC03_MOE_VC07':'PPOV_Adult_U64_MOE',
        'HC03_EST_VC10':'PPOV_Adult_60',
        'HC03_MOE_VC10':'PPOV_Adult_60_MOE',
        'HC03_EST_VC11':'PPOV_Adult_65',
        'HC03_MOE_VC11':'PPOV_Adult_65_MOE',
        'HC03_EST_VC14':'PPOV_Male',
        'HC03_MOE_VC14':'PPOV_Male_MOE',
        'HC03_EST_VC15':'PPOV_Female',
        'HC03_MOE_VC15':'PPOV_Female_MOE',
        'HC03_EST_VC72':'PPOV_Adult_65_74',
        'HC03_MOE_VC72':'PPOV_Adult_65_74_MOE',
        'HC03_EST_VC73':'PPOV_Adult_75',
        'HC03_MOE_VC73':'PPOV_Adult_75_MOE'
        }
        self.df.rename(columns=headers, inplace=True)

        
    def eduHeaders(self):
        """ Currently no change of the headers in the education dataset. Called if niceName == True."""
        
        print('Education does not currently have alternative headers')

        
    def pluHeaders(self):
        """ Change the headers of the plumbing dataset. Called if niceName == True."""
        
        headers = {
        'HD01_VD01':'PLU_Total',
        'HD02_VD01':'PLU_Total_MOE',
        'HD01_VD02':'PLU_Complete',
        'HD02_VD02':'PLU_Complete_MOE',
        'HD01_VD03':'PLU_Lacking',
        'HD02_VD03':'PLU_Lacking_MOE',
        }
        self.df.rename(columns=headers, inplace=True)

        

def main():
    """ Instantiate ACSdata. All parameters can be given during execution as text input.
    
    Assumes data is in the same format as ACS 5-year estimate data for poverty, education or plumbing.   
    Currently requires text inputs from the user for parameters other than booleans. 
    
    Optional parameters, with their default values, descriptions:
        niceName=False, change headers?
        add=False, add County column?
        drop=True, drop the education data columns that are not involved in current analyses this code was made for. 
        

    For niceName, add, desc: unless set to True during instantiation, user will be prompted again.  
    """

    file = ACSdata()
    file.newFile()
    
main()

Enter file name or pathname, including ".csv": /Users/Kelsey/Desktop/GeorgiaContinued/ACS_EDU/ACS_17_5YR_S1501_with_ann.csv
Pathname entered: /Users/Kelsey/Desktop/GeorgiaContinued/ACS_EDU/ACS_17_5YR_S1501_with_ann.csv


/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:535: DtypeWarning: Columns (1,3,4,7,8,11,12,15,16,19,20,23,24,27,28,31,32,35,36,39,40,43,44,47,48,51,52,55,56,59,60,63,64,67,68,71,72,75,76,79,80,83,84,87,88,91,92,95,96,99,100,103,104,107,108,111,112,115,116,119,120,123,124,127,128,131,132,135,136,139,140,143,144,147,148,151,152,155,156,183,184,187,188,191,192,195,196,199,200,203,204,207,208,211,212,215,216,219,220,223,224,227,228,231,232,235,236,239,240,243,244,247,248,251,252,255,256,259,260,263,264,267,268,271,272,275,276,279,280,283,284,287,288,291,292,295,296,299,300,303,304,307,308,311,312,315,316,319,320,323,324,327,328,331,332,335,336,339,340,343,344,347,348,351,352,355,356,359,360,363,364,367,368,371,372,375,376,379,380,383,384,387,388,391,392,395,396,399,400,403,404,407,408,411,412,415,416,419,420,423,424,427,428,431,432,435,436,439,440,443,444,447,448,451,452,455,456,459,460,463,464,467,468,471,472,475,476,479,480,483,484,487,488,491,492,495,496,499,500,503,504,50

Prefer different headers? Enter any character for True. Else press return.t
Different headers is True
which ACS dataset? enter pov, edu,  plu or gen: edu
dataType: edu
Processing ACS education dataset
True
Categories containing the following in their headers were dropped: VC08


/anaconda3/lib/python3.7/site-packages/pandas/core/indexes/base.py:2095: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  result = getitem(key)


min_HS_25, the sum of the columns with the percent over 25 with, at minimum HS attainment, has been added 
Education does not currently have alternative headers
Before removing null values, the length of the dataset is 1969 rows
The 14 rows with >15 null are: [132, 166, 245, 270, 271, 389, 742, 955, 1128, 1150, 1469, 1474, 1512, 1513]
The length of the dataset is now 1955 rows
The remaining 0 columns with >15 null are: []
Add county column? Enter any character for True. Else press return.t
County column added: True
Generate descriptive file? Enter any character for True. Else press return.t
Generate descriptive file: True
Enter descriptive file name or pathname, including ".csv": t.csv
Look in directory for the descriptive file t.csv
Enter output file name or pathname, including ".csv": edu.csv
Look in directory for the output file edu.csv
